In [239]:
!pip install -r requirements.txt 

#import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

def replace_tokens(text, tokenised_text, keywords, punctuation_list, mode):
    keyword_replacements = []
    for i, t in enumerate(keywords):
        target_word = str(t)
        # ignore if it's just punctuation
        if target_word in punctuation_list:
            continue
        # ignore if it's not the list of BERT tokens
        if target_word.lower() not in tokenised_text:
            continue
            
        target_token = target_word.lower()
        masked_index = tokenised_text.index(target_token)
        tokenised_text[masked_index] = '[MASK]'

        indexed_tokens = tokeniser.convert_tokens_to_ids(tokenised_text)
        segments_ids = [1] * len(tokenised_text)
        segments_ids[0] = 0
        segments_ids[1] = 0

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # Predict all tokens
        predictions = model(tokens_tensor, segments_tensors)

        chosen_token = "X"
        while True:
            predicted_index = torch.argmax(predictions[0, masked_index]).item()
            predicted_token = tokeniser.convert_ids_to_tokens([predicted_index])[0]
            if predicted_token not in punctuation_list and predicted_token.find(target_token) < 0 and target_token.find(predicted_token) < 0:
                try:
                    ri = keyword_replacements.index(predicted_token)
                except ValueError:
                    ri = -1
                if predicted_token == 'bell':
                    print("pt: " + predicted_token)
                    print(f'ri: {ri}')
                    print(f'kr: {keyword_replacements[ri]}')
                    print(f'k: {keywords[ri]}')
                    print(f'k: {keywords[i]}')
                if ri < 0 or (ri >= 0 and keywords[i] == keywords[ri]):
                    # and keyword_replacements[ri].find(predicted_token) < 0 and predicted_token.find(keyword_replacements[ri]) < 0)
                    chosen_word = predicted_token
                    #print(f'{target_word} -> {chosen_word}')
                    break
                else:
                    predictions[0,masked_index,predicted_index] = -9999
            else:
                predictions[0,masked_index,predicted_index] = -9999

        if mode == 'all':
            current_text_tokens = spacy_tokeniser(text)
            text = ' '.join([str(spacy_tokeniser(chosen_word)) if str(token) == target_word else str(token) for token in current_text_tokens])
        else:
            text = text.replace(target_word, chosen_word, 1)
            
        tokenised_text = tokeniser.tokenize(text)
        keyword_replacements.append(chosen_word)
        
    return text
    

model_path = "bert-base-uncased"
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

tokeniser = BertTokenizer.from_pretrained(model_path)
#f1 = open("data/hello_world.txt", "r")
#text = "dummy. " + f1.read()
#text = ''
#text = "How are you planning to go the college today?"
#text = "dummy. " + "I am in the college."
#text = "dummy. " + "How are you?"
#tokenised_text = tokeniser.tokenize(text)
a_file = open("data/InputFile.txt","r")
lines = a_file.readlines()

f2 = open("data/keywords.txt", "r")
keywords = f2.read().split()


'''nlp = English()
spacy_tokeniser = nlp.Defaults.create_tokenizer(nlp)
keywords = spacy_tokeniser(text.replace('dummy. ','',1))'''

punctuation = [
    ',',
    '.',
    '!',
    '?',
    ':',
    ';',
    '...',
    '\n', 
    '\n\n',
    '""'
]

file1 = open(r"synthetic_data_MURALI.txt","a")

for k in range(1):   #change the range to get more data points
    generated_lines = []
    final_generated_lines = []
    for i in range((k*10),(k*10) + 10):
        tokenised_text = tokeniser.tokenize(lines[i])
        text = replace_tokens(lines[i], tokenised_text, keywords, punctuation, "selection")
        new_line = (str(text.split('\n')))
    
        generated_lines.append(new_line)
    for j in range(len(generated_lines)):
        final_generated_lines.append(str(''.join(generated_lines[j]).strip('[]')))
        file1.writelines(str(final_generated_lines[j]))
        file1.write("\n")

file1.close()

You should consider upgrading via the 'c:\users\nisha\anaconda3\python.exe -m pip install --upgrade pip' command.
